<a href="https://colab.research.google.com/github/NatashaB-randao/Formacao-Machine-Learning-Specialist/blob/main/ConfusionMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q tensorflow

In [3]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np

import seaborn as sns

import pandas as pd

In [4]:
tf.__version__

'2.17.0'

In [5]:
%load_ext tensorboard

In [6]:
logdir='log'